In [5]:
#Creación y uso del modelo para buscar posibles competidores:


#Importación de librerias y módulos:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

result_df = pd.read_parquet(r'google_ML_recomendaciones.parquet')

In [2]:
#Tomamos un ejemplo:

business_id_to_analyze = '0x89c25c209f53b177:0x8f087a713d995ed6'

In [3]:
#Extraer el estado correspondiente al id_business proporcionado
def obtener_estado_para_negocio(id_negocio, resultado_df=resultado_df):
    if id_negocio not in resultado_df['id_negocio'].values:
        return None  # ID del negocio no encontrado en el conjunto de datos

    estado = resultado_df[resultado_df['id_negocio'] == id_negocio]['estado'].iloc[0]
    return estado

#Para el ejemplo:
estado_para_negocio = obtener_estado_para_negocio(id_negocio_a_analizar)

#Verificar si se encontró el estado
if estado_para_negocio is not None:
    
    #Filtrar resultado_df basado en el estado extraído
    resultado_filtrado_df = resultado_df[resultado_df['estado'] == estado_para_negocio]

    #Preprocesamiento de datos para el marco de datos filtrado
    resultado_filtrado_df['texto'] = resultado_filtrado_df['texto'].fillna('')

    #Se extraen las características para el negocio dado
    estadisticas_negocio_filtrado = resultado_filtrado_df.groupby('id_negocio').agg({
        'calificacion': 'mediana',
        'S_A': 'mediana',
        'texto': ' '.join
    }).reset_index()

    #Vectorización TF-IDF para la columna 'text' en el marco de datos filtrado
    vectorizador_filtrado = TfidfVectorizer(stop_words='english')
    matriz_texto_filtrado = vectorizador_filtrado.fit_transform(estadisticas_negocio_filtrado['texto'])
    
    #Calcular puntajes de similitud para el marco de datos filtrado
    similitud_coseno_filtrado = linear_kernel(matriz_texto_filtrado, matriz_texto_filtrado)

    #Función para obtener recomendaciones para el negocio dado utilizando la similitud_coseno 
    def obtener_recomendaciones_filtradas(id_negocio, similitud_coseno=similitud_coseno_filtrado, estadisticas_negocio=estadisticas_negocio_filtrado):
        if id_negocio not in estadisticas_negocio['id_negocio'].values:
            return "No se encontró el ID del restaurante"

        indice = estadisticas_negocio[estadisticas_negocio['id_negocio'] == id_negocio].index
        if len(indice) == 0:
            return "No se encontró el ID del restaurante"
        
        indice = indice[0]
        
        puntajes_sim = list(enumerate(similitud_coseno[indice]))
        puntajes_sim = sorted(puntajes_sim, key=lambda x: x[1], reverse=True)

        indices_negocio = [x[0] for x in puntajes_sim]
        return estadisticas_negocio['id_negocio'].iloc[indices_negocio]

    recomendaciones_filtradas = obtener_recomendaciones_filtradas(id_negocio_a_analizar)

    #EXcluir el negocio dado si aparece en las recomendaciones
    recomendaciones_filtradas = recomendaciones_filtradas[recomendaciones_filtradas != id_negocio_a_analizar]
    
    #Mostrar el resultado 
    print('¡Encontramos similitudes!')
else:
    print("No se encontró el restaurante.")



C:\Users\Stiven Lopez\AppData\Local\Temp\ipykernel_7108\2117262069.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_result_df['text'] = filtered_result_df['text'].fillna('')


Encontramos recomendaciones!


In [4]:
#Top 5
top_recommendations = recommendations_filtered.head(5)

#Dataframe con el top 5
recommendations_df = pd.DataFrame({'id_business': top_recommendations.values})

merged_df = recommendations_df.merge(result_df[['id_business', 'business_name', 'latitude', 'longitude', 'state', 'category']].drop_duplicates(subset='id_business'),
                                     on='id_business', how='left')

#Mostrar el resultado:
merged_df

,id_business,business_name,latitude,longitude,state,category
0,0x89c25fa9547d17bf:0x404a317c0dd02ce8,Guayaco's,40.746174,-73.883418,New York,"Ecuadorian restaurant, Delivery Restaurant"
1,0x89c25ab32b674bcd:0x56092d9c66754e65,Jalapeños,40.641039,-74.014963,New York,"Mexican restaurant, Delivery Restaurant"
2,0x89c25fafb22b7a59:0xefb0ce999cad815a,12 Corazones,40.747999,-73.879272,New York,"Ecuadorian restaurant, Mexican restaurant"
3,0x89c25fb75e841d45:0x16dee52266222dba,Comida Ecuatoriana,40.748595,-73.870047,New York,Ecuadorian restaurant
4,0x89c25fc977e0a843:0x3f9a21eb8144d281,El Cuencanito,40.743542,-73.862526,New York,"Ecuadorian restaurant, Coffee shop, Latin Amer..."
